A graph is a set of objects called nodes (or vertices) connected by a set of edges (or arcs). If the edges are unidirectional, the graph is called a directed graph or digraph. In a directed graph, if there is an edge from n1 to n2, we refer to n1 as the source or parent node and n2 as the destination or child node.

In [32]:
class Node(object):
    def __init__(self, name):
        """
        :param name: a string
        """
        self.name = name
    def get_name(self):
        return self.name
    def __str__(self):
        return self.name

class Edge(object):
    def __init__(self, src, dest):
        """
        :param src: node
        :param dest: node
        """
        self._src = src
        self._dest = dest
    def get_source(self):
        return self._src
    def get_destination(self):
        return self._dest
    def __str__(self):
        return self._src.get_name() + " -> " + self._dest.get_name()

class Weighted_Edge(Edge):
    def __init__(self, src, dest, weight = 1.0):
        """
        :param src: node
        :param dest: node
        :param weight: a positive number
        """
        super().__init__(src ,dest)
        self._weight = weight
    def get_weight(self):
        return self._weight
    def __str__(self):
        return self.get_source().get_name() + " -> " + self.get_destination().get_name() + " weight: " + str(self._weight)

In [33]:
class Digraph(object):
    """
    nodes is a list of the nodes in the graph
    edges is a dict mapping rach node to a list of it's children
    """
    def __init__(self):
        self._nodes = []
        self._edges = {}
    def add_node(self, node):
        if node in self._nodes:
            raise ValueError("Node already exists")
        else:
            self._nodes.append(node)
            self._edges[node] = []
    def add_edge(self, edge):
        src = edge.get_source()
        dest = edge.get_destination()
        if not (src in self._nodes and dest in self._nodes):
            raise ValueError("Node does not exist")
        self._edges[src].append(dest)
    def children_of(self, node):
        return self._edges[node]
    def has_node(self, node):
        return node in self._nodes
    def __str__(self):
        result = ''
        for src in self._nodes:
            for dest in self._edges[src]:
                result = (result + src.get_name() + " -> " + dest.get_name() + '\n')
        return result[:-1] #omit final newline

class Graph(Digraph):
    def add_edge(self, edge):
        Digraph.add_edge(self, edge)
        rev = Edge(edge.get_destination(), edge.get_source())
        Digraph.add_edge(self, rev)

class WeightedDigraph(object):
    def __init__(self):
        self._nodes = []
        self._edges = {}   # node -> list of Weighted_Edge objects

    def add_node(self, node):
        if node in self._nodes:
            raise ValueError("Node already exists")
        self._nodes.append(node)
        self._edges[node] = []

    def add_edge(self, edge):
        src = edge.get_source()
        dest = edge.get_destination()
        # both must already be nodes
        if not (src in self._nodes and dest in self._nodes):
            raise ValueError("Node does not exist")
        self._edges[src].append(edge)

    def children_of(self, node):
        # return just the destination nodes
        return [e.get_destination() for e in self._edges[node]]

    def get_weight(self, src, dest):
        # look up the weight from src -> dest
        for e in self._edges[src]:
            if e.get_destination() == dest:
                return e.get_weight()
        raise ValueError("Edge does not exist")

    def __str__(self):
        result = ''
        for src in self._nodes:
            for e in self._edges[src]:
                result += str(e) + '\n'
        return result[:-1]  # omit final newline


In [34]:
def print_path(path):
    """
    :param path: a list of nodes
    :return: string
    """
    result = ''
    for node in range(len(path)):
        result += str(path[node])
        if node != len(path) - 1:
            result += '->'
    return result

def DFS(graph, start, end, path, shortest, to_print = False):
    """
    :param graph: Digraph
    :param start: node
    :param end: node
    :param path: list of nodes
    :param shortest: shortest list of nodes
    :param to_print: bool
    :return: a shortest path from start to end
    """
    path = path + [start]
    if to_print:
        print("Current DFS path:", print_path(path))
    if start == end:
        return path
    for node in graph.children_of(start):
        if node not in path: #avoid cycles
            if shortest == None or len(path) < len(shortest):
                new_path = DFS(graph, node, end, path, shortest, to_print)
                if new_path != None:
                    shortest = new_path
    return shortest

def DFS_weighted(graph, start, end,
                 path, total_weight,
                 best_path, best_weight,
                 to_print=False):
    """
    :param graph: WeightedDigraph (or Digraph with get_weight)
    :param start: Node
    :param end: Node
    :param path: list of nodes (current path)
    :param total_weight: sum of weights along current path
    :param best_path: list of nodes (best path found so far)
    :param best_weight: number (weight of best_path)
    :param to_print: bool
    :return: (best_path, best_weight) or (None, None) if no path
    """
    path = path + [start]

    if to_print:
        print("Current DFS path:", print_path(path),
              "with total weight", total_weight)

    # Base case: reached destination
    if start == end:
        return path, total_weight

    for node in graph.children_of(start):
        if node not in path:   # avoid cycles
            w = graph.get_weight(start, node)
            new_total = total_weight + w

            # Prune if we already have a better path
            if best_weight is None or new_total < best_weight:
                candidate_path, candidate_weight = DFS_weighted(
                    graph, node, end,
                    path, new_total,
                    best_path, best_weight,
                    to_print
                )
                if candidate_path is not None:
                    if best_weight is None or candidate_weight < best_weight:
                        best_path, best_weight = candidate_path, candidate_weight

    return best_path, best_weight


def shortest_path(graph, start, end, to_print=False):
    """
    :param graph: Digraph
    :param start: node
    :param end: node
    :param to_print: bool
    :return: shortest path from start to end
    """
    return DFS(graph, start, end, [], None, to_print)

def shortest_path_weighted(graph, start, end, to_print=False):
    """
    :return: (path, total_weight) for minimum-weight path from start to end
    """
    best_path, best_weight = DFS_weighted(
        graph, start, end,
        path=[],
        total_weight=0,
        best_path=None,
        best_weight=None,
        to_print=to_print
    )
    return best_path, best_weight


def BFS(graph, start, end, to_print=False):
    """
    :param graph: Digraph
    :param start: node
    :param end: node
    :param to_print: bool
    :return: the shortest path from start to end
    """
    init_path = [start]
    path_queue = [init_path]
    while len(path_queue) != 0:
        # Get and remove oldest element in path_queue
        tmp_path = path_queue.pop(0)
        if to_print:
            print('Current BFS path:', print_path(tmp_path))
        last_node = tmp_path[-1]
        if last_node == end:
            return tmp_path
        for next_node in graph.children_of(last_node):
            new_path = tmp_path + [next_node]
            path_queue.append(new_path)
    return None

The algorithm above is an example of a recursive depth-first-search (DFS) algorithm

In [35]:
def test_SP():
    nodes = []
    for name in range(6): #create 6 nodes
        nodes.append(Node(str(name)))
    g = Digraph()
    for n in nodes:
        g.add_node(n)
    g.add_edge(Edge(nodes[0], nodes[1]))
    g.add_edge(Edge(nodes[1], nodes[2]))
    g.add_edge(Edge(nodes[2], nodes[3]))
    g.add_edge(Edge(nodes[2], nodes[4]))
    g.add_edge(Edge(nodes[3], nodes[4]))
    g.add_edge(Edge(nodes[3], nodes[5]))
    g.add_edge(Edge(nodes[0], nodes[2]))
    g.add_edge(Edge(nodes[1], nodes[0]))
    g.add_edge(Edge(nodes[3], nodes[1]))
    g.add_edge(Edge(nodes[4], nodes[0]))

    sp = shortest_path(g, nodes[0], nodes[5], to_print=True)
    print('Shortest path found by DFS:', print_path(sp))

def test_SP_weighted():
    nodes = [Node(str(i)) for i in range(6)]
    g = WeightedDigraph()
    for n in nodes:
        g.add_node(n)

    g.add_edge(Weighted_Edge(nodes[0], nodes[1], 2))
    g.add_edge(Weighted_Edge(nodes[1], nodes[2], 5))
    g.add_edge(Weighted_Edge(nodes[2], nodes[3], 1))
    g.add_edge(Weighted_Edge(nodes[2], nodes[4], 4))
    g.add_edge(Weighted_Edge(nodes[3], nodes[5], 3))
    g.add_edge(Weighted_Edge(nodes[0], nodes[2], 10))

    path, total_w = shortest_path_weighted(g, nodes[0], nodes[5], to_print=True)
    print("Best path:", print_path(path))
    print("Total weight:", total_w)


In [36]:
test_SP() #Depth Fist paths

Current DFS path: 0
Current DFS path: 0->1
Current DFS path: 0->1->2
Current DFS path: 0->1->2->3
Current DFS path: 0->1->2->3->4
Current DFS path: 0->1->2->3->5
Current DFS path: 0->1->2->4
Current DFS path: 0->2
Current DFS path: 0->2->3
Current DFS path: 0->2->3->4
Current DFS path: 0->2->3->5
Current DFS path: 0->2->3->1
Current DFS path: 0->2->4
Shortest path found by DFS: 0->2->3->5


In [37]:
nodes = []
for name in range(6): #create 6 nodes
    nodes.append(Node(str(name)))
g = Digraph()
for n in nodes:
    g.add_node(n)
g.add_edge(Edge(nodes[0], nodes[1]))
g.add_edge(Edge(nodes[1], nodes[2]))
g.add_edge(Edge(nodes[2], nodes[3]))
g.add_edge(Edge(nodes[2], nodes[4]))
g.add_edge(Edge(nodes[3], nodes[4]))
g.add_edge(Edge(nodes[3], nodes[5]))
g.add_edge(Edge(nodes[0], nodes[2]))
g.add_edge(Edge(nodes[1], nodes[0]))
g.add_edge(Edge(nodes[3], nodes[1]))
g.add_edge(Edge(nodes[4], nodes[0]))

sp = BFS(g, nodes[0], nodes[5], to_print=True)
print('Shortest path found by BFS:', print_path(sp))


Current BFS path: 0
Current BFS path: 0->1
Current BFS path: 0->2
Current BFS path: 0->1->2
Current BFS path: 0->1->0
Current BFS path: 0->2->3
Current BFS path: 0->2->4
Current BFS path: 0->1->2->3
Current BFS path: 0->1->2->4
Current BFS path: 0->1->0->1
Current BFS path: 0->1->0->2
Current BFS path: 0->2->3->4
Current BFS path: 0->2->3->5
Shortest path found by BFS: 0->2->3->5


In [39]:
test_SP_weighted()

Current DFS path: 0 with total weight 0
Current DFS path: 0->1 with total weight 2
Current DFS path: 0->1->2 with total weight 7
Current DFS path: 0->1->2->3 with total weight 8
Current DFS path: 0->1->2->3->5 with total weight 11
Current DFS path: 0->2 with total weight 10
Best path: 0->1->2->3->5
Total weight: 11
